In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import re
import functools
from time import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import utils
import data_reader
from batch_norm import Batch_Normalizer
from model import CNN_Model

In [2]:
# File parameters
DATA_DIR = 'data'
TRAIN_IMAGES = 'data_train.bin'
TRAIN_TEMPLATES = 'fv_train.bin'
VALID_IMAGES = 'data_valid.bin'
VALID_TEMPLATES = 'fv_valid.bin'
TEST_IMAGES = 'data_test.bin'

# Data parameters
NUM_TRAIN_IMAGES = 100000
NUM_VALID_IMAGES = 10000
NUM_TEST_IMAGES = 10000
IMAGE_DIM = 48
TEMPLATE_DIM = 128

# Batch parameters
BATCH_SIZE = 100
VALID_BATCH_SIZE = 1000

# Batch norm parameters
EMA_DECAY = 0.99
BN_EPSILON = 0.01

# Log parameters
LOG_DIR = 'logs'
SAVE_DIR = 'checkpoints'

In [7]:
def run1():
    model = CNN_Model('config3', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
                      VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

    model.add_conv('conv_1_1', ksize=[3, 3, 1, 10])
    model.add_batch_norm('bn_1_1')
    model.add_conv('conv_1_2', ksize=[3, 3, 10, 10])
    model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_1_2')

    model.add_conv('conv_2_1', ksize=[3, 3, 10, 20])
    model.add_batch_norm('bn_2_1')
    model.add_conv('conv_2_2', ksize=[3, 3, 20, 20])
    model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_2_2')

    model.add_conv('conv_3_1', ksize=[3, 3, 20, 40])
    model.add_batch_norm('bn_3_1')
    model.add_conv('conv_3_2', ksize=[3, 3, 40, 40])
    model.add_pool('avg_pool_3', 'avg', ksize=[1, 6, 6, 1], stride=[1, 6, 6, 1])
    model.add_batch_norm('bn_3_2')

    model.add_fully_connected('fc', size=128)

    model.add_mse_loss('mse_loss')
    model.add_adam_optimizer('optimizer', init_learning_rate=0.005, decay=False)
    model.add_summaries('summaries')

    
    utils.add_description("""
    model = CNN_Model('config3', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
                      VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

    model.add_conv('conv_1_1', ksize=[3, 3, 1, 10])
    model.add_batch_norm('bn_1_1')
    model.add_conv('conv_1_2', ksize=[3, 3, 10, 10])
    model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_1_2')

    model.add_conv('conv_2_1', ksize=[3, 3, 10, 20])
    model.add_batch_norm('bn_2_1')
    model.add_conv('conv_2_2', ksize=[3, 3, 20, 20])
    model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_2_2')

    model.add_conv('conv_3_1', ksize=[3, 3, 20, 40])
    model.add_batch_norm('bn_3_1')
    model.add_conv('conv_3_2', ksize=[3, 3, 40, 40])
    model.add_pool('avg_pool_3', 'avg', ksize=[1, 6, 6, 1], stride=[1, 6, 6, 1])
    model.add_batch_norm('bn_3_2')

    model.add_fully_connected('fc', size=128)

    model.add_mse_loss('mse_loss')
    model.add_adam_optimizer('optimizer', init_learning_rate=0.005, decay=False)
    model.add_summaries('summaries')

    total parameters = %d
    """  % (utils.num_parameters(model)), model.save_dir, model.name)
    
    
    model.initialize_session(restore=True)
    model.train(n_batches=10000, step_size=500, batch_size=100, valid_batch_size=1000, save=True)
    model.close_session()

In [8]:
run1()

Description added at checkpoints\config3\config3_description.txt
Restored session checkpoints\config3\model.ckpt-28000
Batch 28500:
--> Train loss = 0.006013
--> Valid loss = 0.006749
Saving session checkpoints\config3\model.ckpt-28500
Batch 29000:
--> Train loss = 0.006000
--> Valid loss = 0.006762
Saving session checkpoints\config3\model.ckpt-29000
Batch 29500:
--> Train loss = 0.005993
--> Valid loss = 0.006770
Saving session checkpoints\config3\model.ckpt-29500
Batch 30000:
--> Train loss = 0.005998
--> Valid loss = 0.006806
Saving session checkpoints\config3\model.ckpt-30000
Batch 30500:
--> Train loss = 0.005992
--> Valid loss = 0.006773
Saving session checkpoints\config3\model.ckpt-30500
Batch 31000:
--> Train loss = 0.005984
--> Valid loss = 0.006765
Saving session checkpoints\config3\model.ckpt-31000
Batch 31500:
--> Train loss = 0.005984
--> Valid loss = 0.006727
Saving session checkpoints\config3\model.ckpt-31500
Batch 32000:
--> Train loss = 0.005982
--> Valid loss = 0.0067

# Predictions

In [10]:
# model = CNN_Model('config6', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
#                   VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

# model.add_conv('conv_1', ksize=[3, 3, 1, 16])
# model.add_pool('pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
# model.add_batch_norm('bn_1')

# model.add_conv('conv_2', ksize=[3, 3, 16, 32])
# model.add_pool('pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
# model.add_batch_norm('bn_2')

# model.add_conv('conv_3', ksize=[3, 3, 32, 32])
# model.add_pool('pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
# model.add_batch_norm('bn_3')

# model.add_conv('conv_4', ksize=[3, 3, 32, 32])
# model.add_pool('pool_4', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
# model.add_batch_norm('bn_4')

# model.add_conv('conv_5', ksize=[3, 3, 32, 62])
# model.add_pool('pool_5', 'max', ksize=[1, 3, 3, 1], stride=[1, 3, 3, 1])
# model.add_batch_norm('bn_5')

# model.add_fully_connected('fc', size=128)

# model.add_mse_loss('mse_loss')
# model.add_adam_optimizer('optimizer', init_learning_rate=0.0005, decay=False)
# model.add_summaries('summaries')


# test_images, _ = data_reader.load_data(DATA_DIR, TEST_IMAGES, VALID_TEMPLATES, NUM_TEST_IMAGES, IMAGE_DIM, TEMPLATE_DIM)
# predictions = model.predict(test_images, batch_size=1000)

# f = open('data/template_pred_config6.bin', 'wb')
# for i in range(NUM_TEST_IMAGES):
#     f.write(predictions[i, :])
# f.close()